In [51]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StansdardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import gower

In [88]:
def KNNImputerProcess(path, max_min=None):
    df = pd.read_csv(path)
    id = df["id"]
    df = df.drop(["id", "height"], axis=1)

    #bmi
    for i in range(len(df['bmi'])):
        if df['bmi'][i] >= 45:
            df['bmi'][i] = 30.0
    
    #marriage
    marriage_mapping = {'No': 0, 'Yes': 1}
    df['ever_married'] = df['ever_married'].map(marriage_mapping)

    #one_hot encoder for ...
    categorical_features_oh = ['gender', 'blood', 'smoking_status', 'Residence_type', 'work_type']
    for i in categorical_features_oh:
        df[i] = pd.Categorical(df[i])
        dfDummies = pd.get_dummies(df[i], prefix= i)
        df = pd.concat([df, dfDummies], axis=1)
    for i in categorical_features_oh:
        df.drop(columns=i, axis=1, inplace=True)
    
    flag = False
    #normalize
    if max_min == None:
        flag = True
        max_min = {}
        norm_feature = ['age', 'avg_glucose_level', 'bmi']
        for i in norm_feature:
            max = np.nanmax(df[i].to_numpy())
            min = np.nanmin(df[i].to_numpy())
            max_min[i] = [max, min]
            n = max - min
            for j in range(len(df[i])):
                df[i][j] = (df[i][j] - min)/n
    else:
        norm_feature = ['age', 'avg_glucose_level', 'bmi']
        for i in norm_feature:
            max = max_min[i][0]
            min = max_min[i][1]
            n = max - min
            for j in range(len(df[i])):
                if df[i][j] < min:
                    df[i][j] = 0
                elif df[i][j] > max:
                    df[i][j] = 1
                else: df[i][j] = (df[i][j] - min)/n
    
    col = df.columns
    imputer = KNNImputer(n_neighbors=20)
    imputer.fit(df)
    df = imputer.transform(df)
    df = pd.DataFrame(df, columns=col)

    df = pd.concat([df, id], axis=1)
    if flag == True:
        return df, max_min
    else: return df

In [78]:
df_train, max_min = KNNImputerProcess('./Dataset/train.csv')
df_train = df_train.drop(['id'], axis=1)

<ipython-input-77-5d52ce94a3b9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bmi'][i] = 30.0
<ipython-input-77-5d52ce94a3b9>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i][j] = (df[i][j] - min)/n


In [79]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   age                             4000 non-null   float64
 1   hypertension                    4000 non-null   float64
 2   heart_disease                   4000 non-null   float64
 3   ever_married                    4000 non-null   float64
 4   avg_glucose_level               4000 non-null   float64
 5   bmi                             4000 non-null   float64
 6   stroke                          4000 non-null   float64
 7   gender_Female                   4000 non-null   float64
 8   gender_Male                     4000 non-null   float64
 9   gender_Other                    4000 non-null   float64
 10  blood_A                         4000 non-null   float64
 11  blood_AB                        4000 non-null   float64
 12  blood_B                         40

MODEL

In [80]:
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectFromModel
import lightgbm as gbm

In [81]:
x = df_train.drop(["stroke"], axis=1).to_numpy()
y = df_train["stroke"].to_numpy()

In [82]:
svm_smt = SVMSMOTE()
tl = TomekLinks(sampling_strategy='majority')
resample = Pipeline([('smt', svm_smt), ('tl', tl)])
x_new, y_new = resample.fit_resample(x, y)

In [83]:
model = gbm.LGBMClassifier(boosting='gbdt', max_depth=-1, n_estimators=500)
model.fit(x_new, y_new)
thresholds = np.sort(model.booster_.feature_importance())
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
scoring = ['f1']
scores = cross_validate(model, x, y, scoring=scoring, cv=cv, n_jobs=-1)
max = np.mean(scores['test_f1'])
best_transform = None
best_model = model
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(x_new)
    # train model
    selection_model = gbm.LGBMClassifier(boosting='gbdt', max_depth=-1, n_estimators=500)
    selection_model.fit(select_X_train, y_new)
	# eval model
    scores2 = cross_validate(selection_model, select_X_train, y_new, scoring=scoring, cv=cv, n_jobs=-1)
    print("Thresh=", (thresh))
    print(np.mean(scores2['test_f1']))
    if np.mean(scores2['test_f1']) > max:
        max = np.mean(scores2['test_f1'])
        best_model = selection_model
        best_transform = selection

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Thresh= 0
0.9876024948080249
Thresh= 0
0.9876024948080249
Thresh= 100
0.9876024948080249
Thresh= 108
0.9870866737703224
Thresh= 201
0.9873447624546108
Thresh= 201
0.9873447624546108
Thresh= 212
0.9873513754330034
Thresh= 219
0.9870999615970855
Thresh= 220
0.9881166570809569
Thresh= 223
0.9883773695203336
Thresh= 232
0.9879862824728569
Thresh= 254
0.9888767968330562
Thresh= 266
0.9893979643474863
Thresh= 285
0.9881233172728144
Thresh= 289
0.986826582087876
Thresh= 291
0.9850140510513354
Thresh= 298
0.9856531656363217
Thresh= 325
0.9828104326233605
Thresh= 353
0.9764928444858881
Thresh= 419
0.9768520226255071
Thresh= 495
0.9710898389524267
Thresh= 1040
0.9644000520366923
Thresh= 2444
0.9522375612135209
Thresh= 2574
0.9449786474441136
Thresh= 2650
0.9382947807071398


In [84]:
if best_transform != None:
    x_new = best_transform.transform(x_new)

In [13]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
# define models and parameters
boosting = ['gbdt', 'rf', 'dart', 'goss']
n_estimators = [10, 100, 500, 1000]
max_depth = [2, 5, 10, -1]
# define grid search
grid = dict(boosting=boosting, n_estimators=n_estimators, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1',error_score=0)
grid_result = grid_search.fit(x_new, y_new)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
Best: 0.989636 using {'boosting': 'gbdt', 'max_depth': 10, 'n_estimators': 500}
0.920903 (0.011280) with: {'boosting': 'gbdt', 'max_depth': 2, 'n_estimators': 10}
0.964387 (0.005425) with: {'boosting': 'gbdt', 'max_depth': 2, 'n_estimators': 100}
0.982144 (0.005239) with: {'boosting': 'gbdt', 'max_depth': 2, 'n_estimators': 500}
0.984152 (0.004459) with: {'boosting': 'gbdt', 'max_depth': 2, 'n_estimators': 1000}
0.958044 (0.006858) with: {'boosting': 'gbdt', 'max_depth': 5, 'n_estimators': 10}
0.985531 (0.004766) with: {'boosting': 'gbdt', 'max_depth': 5, 'n_estimators': 100}
0.988901 (0.004704) with: {'boosting': 'gbdt', 'max_depth': 5, 'n_estimators': 500}
0.988898 (0.004593) with: {'boosting': 'gbdt', 'max_depth': 5, 'n_estimators': 1000}
0.975312 (0.005185) with: {'boosting': 'gbdt', 'max_depth': 10, 'n_estimators': 10}
0.987680 (0.004512) with: {'boosting': 'gbdt', 'max_dept

In [21]:
best_model.booster_.feature_importance()

array([2356, 1071,  580,  187, 2396, 2293,  211,  378,   94,  186,  270,
        178,  339,  250,  388,  402,  336,  317,  189,  227,  150,    0,
        242,  278,    0])

In [85]:
best_model = gbm.LGBMClassifier(boosting='gbdt', max_depth=-1, n_estimators=500)

In [86]:
best_model.fit(x_new, y_new)

LGBMClassifier(boosting='gbdt', n_estimators=500)

In [89]:
df_test = KNNImputerProcess('./Dataset/public_test.csv', max_min)

<ipython-input-88-eb82f7d5953f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bmi'][i] = 30.0
<ipython-input-88-eb82f7d5953f>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: df[i][j] = (df[i][j] - min)/n


In [90]:
id_test = df_test['id'].to_numpy()
x_test = df_test.drop(['id'], axis=1).to_numpy()
x_test = np.array(x_test)

In [91]:
if best_transform != None:
    x_test = best_transform.transform(x_test)

In [92]:
y_xgb_pred_test = best_model.predict(x_test)
data_test_submit = {'id': id_test, 'stroke': y_xgb_pred_test}
df_test_submit = pd.DataFrame(data= data_test_submit)
df_test_submit
df_test_submit.to_csv('./Submission/Submission_lgbm.csv', index= False)

In [94]:
x_test.shape

(500, 13)